In [8]:
%env BLINKA_FT232H=1

env: BLINKA_FT232H=1


In [9]:
import os
import time
os.environ["BLINKA_FT232H"]

'1'

In [10]:
from pyftdi.ftdi import Ftdi
# print("Detected devices:", Ftdi.list_devices())
Ftdi().open_from_url('ftdi:///?')


Available interfaces:
  ftdi://ftdi:232h:01/1  (USB <-> Serial Converter)



SystemExit: Please specify the USB device

/home/haoran/anaconda3/envs/flaresim/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
from pyftdi.i2c import I2cController
from pyftdi.ftdi import Ftdi

# Print device descriptors to verify uniqueness
print("Detected FTDI devices:")
print(Ftdi.list_devices())

# Create an I2C controller instance for each FT232H device
i2c1 = I2cController()

#   First device (serial '01'):
i2c1.configure('ftdi://ftdi:232h:01/1')

# Get the I2C port for sensor communication (assuming all sensors are at 0x76)
port1 = i2c1.get_port(0x76)




Detected FTDI devices:
[(UsbDeviceDescriptor(vid=1027, pid=24596, bus=5, address=39, sn='01', index=None, description='USB <-> Serial Converter'), 1)]


In [12]:
import board
import digitalio
import keyboard
import ms5837
import busio
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
 

In [13]:
led_aa = digitalio.DigitalInOut(board.D7)
led_aa.direction = digitalio.Direction.OUTPUT
i= 0
while i < 2:
    i += 1
    led_aa.value = True
    print("LED ON")
    time.sleep(0.5)
    led_aa.value = False
    print("LED OFF")
    time.sleep(0.5)

LED ON
LED OFF
LED ON
LED OFF


In [8]:
led_aa.value = False
print("LED OFF")

LED OFF


In [7]:
# Initialize the I²C bus via FT232H 
# (make sure BLINKA_FT232H=1 in your environment)
i2c = busio.I2C(board.SCL, board.SDA)
while not i2c.try_lock():
    pass

sensor_address = 0x76  #check sensor’s address

try:
    # Send reset command 0x1E
    i2c.writeto(sensor_address, bytes([0x1E]))
    time.sleep(0.05)  # wait 50ms for reset
    
    # Read first PROM register (0xA0, returns 2 bytes), check sensor’s datasheet
    i2c.writeto(sensor_address, bytes([0xA0]))
    buffer = bytearray(2)
    i2c.readfrom_into(sensor_address, buffer)
    coeff = (buffer[0] << 8) | buffer[1]
    print(f"PROM coefficient: {coeff}")
    
except Exception as e:
    print("Error communicating with sensor:", e)
finally:
    i2c.unlock()

PROM coefficient: 16385


In [8]:
#!/usr/bin/env python3
import time
import board
import busio

# Initialize the I²C bus (FT232H via BLINKA)
i2c = busio.I2C(board.SCL, board.SDA)
while not i2c.try_lock():
    pass

sensor_address = 0x76  # Sensor I²C address

# Calibration coefficients read from the sensor’s PROM.
# (Typical example values as provided in the datasheet)
C1 = 40127  
C2 = 36924
C3 = 23317
C4 = 23282
C5 = 33464
C6 = 28312

# ------------------- PRESSURE CONVERSION (D1) -------------------
print("Starting pressure conversion...")
# Send the conversion command for pressure (D1) with OSR=4096.
i2c.writeto(sensor_address, bytes([0x48]))
time.sleep(0.02)  # Wait ~20 ms for conversion

# Send ADC read command to obtain a 24-bit result.
i2c.writeto(sensor_address, bytes([0x00]))
raw_pressure_bytes = bytearray(3)
i2c.readfrom_into(sensor_address, raw_pressure_bytes)
D1 = (raw_pressure_bytes[0] << 16) | (raw_pressure_bytes[1] << 8) | raw_pressure_bytes[2]
print("Raw pressure (D1):", D1)

# ------------------- TEMPERATURE CONVERSION (D2) -------------------
print("Starting temperature conversion...")
# Send the conversion command for temperature (D2) with OSR=4096.
i2c.writeto(sensor_address, bytes([0x58]))
time.sleep(0.02)  # Wait ~20 ms for conversion

# Send ADC read command to obtain a 24-bit result.
i2c.writeto(sensor_address, bytes([0x00]))
raw_temp_bytes = bytearray(3)
i2c.readfrom_into(sensor_address, raw_temp_bytes)
D2 = (raw_temp_bytes[0] << 16) | (raw_temp_bytes[1] << 8) | raw_temp_bytes[2]
print("Raw temperature (D2):", D2)

# ------------------- FIRST ORDER CALCULATION -------------------
# Calculate temperature difference (dT)
dT = D2 - (C5 * 256)
# Calculate initial temperature (in hundredths of °C)
TEMP = 2000 + (dT * C6) / 8388608.0   # 2^23 = 8388608
# Calculate offset and sensitivity
OFF = C2 * 65536 + (C4 * dT) / 128.0    # 2^16 = 65536, 2^7 = 128
SENS = C1 * 32768 + (C3 * dT) / 256.0   # 2^15 = 32768, 2^8 = 256

# ------------------- SECOND ORDER COMPENSATION -------------------
# Apply second order corrections when temperature is below 20°C.
if TEMP < 2000:  
    T2 = (dT * dT) / 2147483648.0  # 2^31 = 2147483648
    OFF2 = 5 * ((TEMP - 2000) ** 2) / 2.0
    SENS2 = 5 * ((TEMP - 2000) ** 2) / 4.0
    # Further compensation for very low temperatures (< -15°C)
    if TEMP < -1500:
        OFF2 += 7 * ((TEMP + 1500) ** 2)
        SENS2 += (11 * ((TEMP + 1500) ** 2)) / 2.0
else:
    T2 = 0
    OFF2 = 0
    SENS2 = 0

# Apply the compensation corrections
TEMP = TEMP - T2
OFF = OFF - OFF2
SENS = SENS - SENS2

# Calculate pressure (in hundredths of mbar)
pressure = ((D1 * SENS) / 2097152.0 - OFF) / 32768.0   # 2^21 = 2097152, 2^15 = 32768

# ------------------- OUTPUT RESULTS -------------------
# Divide by 100 to get °C and mbar respectively.
print("Calculated temperature: {:.2f} °C".format(TEMP / 100.0))
print("Calculated pressure: {:.2f} mbar".format(pressure / 100.0))

i2c.unlock()


Starting pressure conversion...
Raw pressure (D1): 6408037
Starting temperature conversion...
Raw temperature (D2): 8576849
Calculated temperature: 20.34 °C
Calculated pressure: 487.93 mbar


In [11]:
# This is for 02BA
# Initialize the I²C bus (FT232H via BLINKA)
i2c = busio.I2C(board.SCL, board.SDA)
while not i2c.try_lock():
    pass

sensor_address = 0x76  # Update if your sensor uses a different address

# Assume that you've already reset the sensor and read calibration coefficients.
# the following coefficients from the datasheet's PROM (values are typical): range(0,65535)
C1 = 40127  
C2 = 36924
C3 = 23317
C4 = 23282
C5 = 33464
C6 = 28312


#C1 = 34982   
# #  C1 = Pressure sensitivity
# C2 = 36352
# #  C2 = Pressure offset
# C3 = 20328 
# #  C3 = Temperature coefficient of pressure sensitivity
# C4 = 22354  
# #  C4 = Temperature coefficient of pressure offset
# C5 = 26646  
# #  C5 = Reference temperature
# C6 = 26146  
# #  C6 = Temperature coefficient of the temperature.


# --- Get Raw Pressure (D1) ---
# Send the command for pressure conversion.
# 0x48 corresponds to Convert D1 with OSR=4096.
print("Starting pressure conversion...")
i2c.writeto(sensor_address, bytes([0x48])) 
# Wait the required conversion time (about 10-20ms for OSR=4096; adjust as needed)
time.sleep(0.02)

# Request the ADC result by sending the ADC read command (0x00)
i2c.writeto(sensor_address, bytes([0x00]))
# Read 3 bytes from the sensor.
raw_pressure_bytes = bytearray(3)
i2c.readfrom_into(sensor_address, raw_pressure_bytes)
# Combine 3 bytes into a 24-bit number.
D1 = (raw_pressure_bytes[0] << 16) | (raw_pressure_bytes[1] << 8) | raw_pressure_bytes[2]
print("Raw pressure (D1):", D1)

# --- Get Raw Temperature (D2) ---
# Send the command for temperature conversion.
# 0x58 corresponds to Convert D2 with OSR=4096.
print("Starting temperature conversion...")
i2c.writeto(sensor_address, bytes([0x58]))
# Wait the required conversion time (again, adjust delay as recommended in the datasheet)
time.sleep(0.02)

# Request the ADC result with 0x00 command.
i2c.writeto(sensor_address, bytes([0x00]))
raw_temp_bytes = bytearray(3)
i2c.readfrom_into(sensor_address, raw_temp_bytes)
D2 = (raw_temp_bytes[0] << 16) | (raw_temp_bytes[1] << 8) | raw_temp_bytes[2]
print("Raw temperature (D2):", D2)

# --- Calculation of Actual Temperature and Pressure ---
#
# The formulas taken from the MS5837 datasheet are typically:
#
#   dT    = D2 - C5 * 2^8
#   TEMP  = 2000 + dT * C6 / 2^23
#
#   OFF   = C2 * 2^16 + (C4 * dT) / 2^7
#   SENS  = C1 * 2^15 + (C3 * dT) / 2^8
#
#   Pressure = (D1 * SENS / 2^21 - OFF) / 2^15
#
dT = D2 - (C5 * 256)
TEMP = 2000 + (dT * C6) / 8388608.0   # 2^23 = 8388608
OFF = C2 * 65536 + (C4 * dT) / 128.0     # 2^16 = 65536, 2^7 = 128
SENS = C1 * 32768 + (C3 * dT) / 256.0    # 2^15 = 32768, 2^8 = 256
pressure = ((D1 * SENS) / 2097152.0 - OFF) / 32768.0   # 2^21 = 2097152, 2^15 = 32768

# The computed temperature is typically in hundredths of degrees C
print("Calculated temperature: {:.2f} °C".format(TEMP / 100.0))
# And pressure might be in hundredths of mbar (or hPa)
print("Calculated pressure: {:.2f} mbar".format(pressure / 100.0))

i2c.unlock()


Starting pressure conversion...
Raw pressure (D1): 4322211
Starting temperature conversion...
Raw temperature (D2): 7444553
Calculated temperature: -17.88 °C
Calculated pressure: 86.54 mbar


In [16]:
import ms5837_app
ms5837_app.main()

AttributeError: module 'ms5837_app' has no attribute 'main'